<a href="https://colab.research.google.com/github/Lakshmi-Nair16/Portfolio/blob/main/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQL Murder Mystery

In [ ]:
import sqlite3 as sql
import pandas as pd

In [ ]:
con = sql. connect('/content/sql-murder-mystery.db')
pd.set_option('display.max_colwidth', None)

We know that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018,​ and that it took place in ​SQL City​.

In [ ]:
query1 = '''
SELECT *
FROM crime_scene_report
WHERE date = '20180115' AND city = 'SQL City';
'''
pd.read_sql_query(query1, con)

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot him in the side! Yes he yields!",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


The report mentions two witnesses: one residing on "Northwestern Dr" and the other named Annabel living on "Franklin Ave."

In [ ]:
query2 = '''
SELECT *
FROM Person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number desc;
'''
pd.read_sql_query(query2,con)[:1]

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


This query searches for the witness who lives on "Northwestern Dr" street and order the results by address number in descending order, helping us identify the exact location of the first witness.


In [ ]:
query3 = '''
SELECT *
FROM Person
WHERE address_street_name = "Franklin Ave" AND name like '%Annabel%';
'''
pd.read_sql_query(query3,con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Similarly, we located the second witness mentioned in the crime scene report, who is named "Annabel" and resides on "Franklin Ave" street.

With both witnesses identified, our next step is to review their interviews for any additional clues.

In [ ]:
query4 = '''
SELECT *
FROM interview
WHERE person_id = 14887 OR person_id = 16371;
'''
pd.read_sql_query(query4,con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Now, let's follow the clues provided in the witness interviews and check the gym check-ins for potential suspects.

In [ ]:
query5 = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id like '%48Z%' AND check_in_date = 20180109
ORDER By check_in_date;
'''
pd.read_sql_query(query5,con)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


We've gotten two members with membership_ids 48Z7A and 48Z55.
We'll verify the gender and car plate information of potential suspects to gather more clues.

In [ ]:
query6 = '''
SELECT *
FROM drivers_license
WHERE plate_number like "%H42W%" AND gender = "male";
'''
pd.read_sql_query(query6,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


With potential suspects identified, let's gather additional details about them from the person table.

In [ ]:
query7 = '''
SELECT *
FROM Person
WHERE license_id ="423327" OR license_id ="664760";
'''
pd.read_sql_query(query7,con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Now, let's check which of these two are a member of the gym.

In [ ]:
query8 = '''
SELECT *
FROM get_fit_now_member
WHERE person_id ="51739" OR person_id ="67318";
'''
pd.read_sql_query(query8,con)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


We've found the murderer - it's Jeremy Bowers

In [ ]:
query_9 = '''
SELECT *
FROM interview
WHERE person_id = 67318;
'''
pd.read_sql_query(query_9,con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


We'll search for individuals matching the description provided by the murderer.

In [ ]:
query10 = '''
SELECT *
FROM drivers_license
WHERE car_make = "Tesla" AND car_model = "Model S" AND gender = "female" AND hair_color = "red";
'''
pd.read_sql_query(query10,con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


Three women fit the criteria.

In [ ]:
query11 = '''
SELECT *
FROM Person
WHERE license_id ="202298" OR license_id ="291182" OR license_id ="918773";
'''
pd.read_sql_query(query11,con)

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


Now, we check who attended the SQL Symphony Concert 3 times.

In [ ]:
query12 = '''
SELECT person_id, event_name, COUNT(*) AS event_count
FROM facebook_event_checkin
WHERE person_id IN ("78881", "90700", "99716")
GROUP BY person_id, event_name;
'''
pd.read_sql_query(query12,con)

,person_id,event_name,event_count
0,99716,SQL Symphony Concert,3


We've found that Miranda Priestly is the REAL VILLAIN.

#**Documentation**
Crime Details:

    *Murder occurred on January 15, 2018, in SQL City.
    *Two key witnesses: one residing on Northwestern Dr and another named Annabel on Franklin Ave.

Investigation Process:

    *Identified witnesses using their addresses (queries 2 and 3).
    *Analyzed witness interviews (query 4) to gather clues.
    *Investigated gym check-ins for potential suspects (query 5).
    *Verified suspect information using drivers' licenses and personal details (queries 6 and 7).
    *Determined gym membership status of potential suspects (query 8).
    *Identified the murderer as Jeremy Bowers (query 9).
    *Searched for individuals matching the murderer's description (query 10 and 11).
    *Identified Miranda Priestly as the mastermind based on concert attendance (query 12).

Key Findings:

    *Jeremy Bowers is the murderer.
    *Miranda Priestly is the mastermind behind the murder.
  
Reflect on the Investigation

The SQL Murder Mystery proved to be a challenging yet rewarding experience. Utilizing SQL to analyze data and draw conclusions was a valuable learning experience.

Key challenges included:

    Interpreting witness statements and extracting relevant information.
    Effectively combining information from different tables to identify suspects.

Overcoming these challenges required careful analysis, logical reasoning, and attention to detail.

Overall, this investigation enhanced my SQL skills and problem-solving abilities.